In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atrial_Fibrillation"
cohort = "GSE235307"

# Input paths
in_trait_dir = "../../input/GEO/Atrial_Fibrillation"
in_cohort_dir = "../../input/GEO/Atrial_Fibrillation/GSE235307"

# Output paths
out_data_file = "../../output/preprocess/Atrial_Fibrillation/GSE235307.csv"
out_gene_data_file = "../../output/preprocess/Atrial_Fibrillation/gene_data/GSE235307.csv"
out_clinical_data_file = "../../output/preprocess/Atrial_Fibrillation/clinical_data/GSE235307.csv"
json_path = "../../output/preprocess/Atrial_Fibrillation/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression and atrial fibrillation prediction"
!Series_summary	"The aim of this study was to identify a blood gene expression profile that predicts atrial fibrillation in heart failure patients"
!Series_overall_design	"Cardiac blood samples were obtained from the coronary sinus during CRT-D (Cardiac Resynchronization Therapy - Defibrillator) placement in heart failure patients. Patients were followed during 1 year."
Sample Characteristics Dictionary:
{0: ['tissue: Whole blood'], 1: ['gender: Male', 'gender: Female'], 2: ['age: 63', 'age: 60', 'age: 72', 'age: 66', 'age: 70', 'age: 64', 'age: 61', 'age: 44', 'age: 54', 'age: 50', 'age: 79', 'age: 51', 'age: 55', 'age: 67', 'age: 52', 'age: 73', 'age: 76', 'age: 43', 'age: 68', 'age: 78', 'age: 69', 'age: 57', 'age: 59', 'age: 53', 'age: 65', 'age: 56', 'age: 74', 'age: 38', 'age: 71', 'age: 37'], 3: ['cardiopathy: ischemic', 'cardiopathy: non ischemic', 'cardiopathy: mixed'], 4: ['cardiac rhyt

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# This dataset appears to be about gene expression in blood samples from heart failure patients
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify keys in the sample characteristics dictionary

# For trait (Atrial_Fibrillation): appears in sample characteristics key 5
# We can see patients had either "Sinus rhythm" or "Atrial fibrillation" after 1 year follow-up
trait_row = 5

# For age: appears in sample characteristics key 2
age_row = 2

# For gender: appears in sample characteristics key 1
gender_row = 1

# 2.2 Define conversion functions

def convert_trait(value):
    """Convert trait value to binary (0 or 1)"""
    if not isinstance(value, str):
        return None
    
    value = value.lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "atrial fibrillation" in value:
        return 1
    elif "sinus rhythm" in value:
        return 0
    return None

def convert_age(value):
    """Convert age value to continuous numeric"""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if not isinstance(value, str):
        return None
    
    value = value.lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# 3. Save metadata for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Assume clinical_data variable exists from previous step
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of extracted clinical features:")
    print(preview_df(clinical_features))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features to CSV file
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM7498589': [0.0, 63.0, 1.0], 'GSM7498590': [0.0, 60.0, 1.0], 'GSM7498591': [0.0, 60.0, 1.0], 'GSM7498592': [0.0, 72.0, 1.0], 'GSM7498593': [0.0, 63.0, 1.0], 'GSM7498594': [0.0, 66.0, 0.0], 'GSM7498595': [0.0, 70.0, 1.0], 'GSM7498596': [0.0, 64.0, 1.0], 'GSM7498597': [0.0, 63.0, 1.0], 'GSM7498598': [0.0, 61.0, 1.0], 'GSM7498599': [0.0, 70.0, 0.0], 'GSM7498600': [0.0, 64.0, 1.0], 'GSM7498601': [0.0, 63.0, 1.0], 'GSM7498602': [0.0, 44.0, 1.0], 'GSM7498603': [0.0, 54.0, 1.0], 'GSM7498604': [0.0, 44.0, 1.0], 'GSM7498605': [0.0, 50.0, 1.0], 'GSM7498606': [1.0, 79.0, 1.0], 'GSM7498607': [0.0, 63.0, 1.0], 'GSM7498608': [0.0, 63.0, 0.0], 'GSM7498609': [1.0, 64.0, 1.0], 'GSM7498610': [0.0, 60.0, 1.0], 'GSM7498611': [0.0, 51.0, 1.0], 'GSM7498612': [0.0, 55.0, 1.0], 'GSM7498613': [0.0, 55.0, 1.0], 'GSM7498614': [1.0, 67.0, 1.0], 'GSM7498615': [0.0, 52.0, 1.0], 'GSM7498616': [0.0, 70.0, 0.0], 'GSM7498617': [0.0, 54.0, 1.0], 'GSM7498618': [0.0, 54.0, 1.0],

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atrial_Fibrillation/GSE235307/GSE235307_series_matrix.txt.gz


Gene data shape: (58717, 119)
First 20 gene/probe identifiers:
Index(['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers '4', '5', '6', etc. are numeric values that do not correspond to human gene symbols
# These appear to be row indices or probe IDs that need to be mapped to actual gene symbols
# In human genomics, gene symbols would typically be alphanumeric identifiers like "BRCA1", "TP53", etc.

# Since these are numeric identifiers and not recognizable gene symbols,
# they will require mapping to standard gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'COL', 'ROW', 'NAME', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'LOCUSLINK_ID', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 

Column 'ID' contains values matching gene data ID pattern


Column 'COL' contains values matching gene data ID pattern


Column 'ROW' contains values matching gene data ID pattern


Column 'NAME' contains values matching gene data ID pattern


...

Column 'CHROMOSOMAL_LOCATION' contains values matching gene data ID pattern


Column 'CYTOBAND' contains values matching gene data ID pattern


Column 'DESCRIPTION' contains values matching gene data ID pattern


Column 'GO_ID' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'NAME' may contain gene-related information
Sample values: ['GE_BrightCorner', 'DarkCorner', 'DarkCorner']
Column 'GENE_SYMBOL' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'GENE_NAME' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'UNIGENE_ID' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'DESCRIPTION' may contain gene-related information
Sample values: [nan, nan, nan]


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the key columns for mapping
prob_col = 'ID'  # The numeric identifiers in gene_data (4, 5, 6, etc.)
gene_col = 'GENE_SYMBOL'  # The gene symbols (HEBP1, KCNE4, etc.)

# Let's verify the structure of our gene annotation dataframe
print("\nSample values in the mapping columns:")
print(f"Probe column '{prob_col}' values: {gene_annotation[prob_col].head().tolist()}")
print(f"Gene column '{gene_col}' values: {gene_annotation[gene_col].head().tolist()}")

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"\nGene mapping shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nGene expression data after mapping, shape: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5].tolist())

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")



Sample values in the mapping columns:
Probe column 'ID' values: ['1', '2', '3', '4', '5']
Gene column 'GENE_SYMBOL' values: [nan, nan, nan, 'HEBP1', 'KCNE4']



Gene mapping shape: (54295, 2)
Gene mapping preview:
{'ID': ['4', '5', '6', '7', '8'], 'Gene': ['HEBP1', 'KCNE4', 'BPIFA3', 'LOC100129869', 'IRG1']}



Gene expression data after mapping, shape: (20353, 119)
First 5 gene symbols after mapping:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M']



Gene expression data saved to ../../output/preprocess/Atrial_Fibrillation/gene_data/GSE235307.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to atrial fibrillation after cardiac surgery, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for atrial fibrillation after cardiac surgery, which is relevant to arrhythmia research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for arrhythmia analysis."
    )
    
    print("Dataset is not usable for arrhythmia analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (20353, 119)
Gene data shape after normalization: (19847, 119)


Normalized gene expression data saved to ../../output/preprocess/Atrial_Fibrillation/gene_data/GSE235307.csv


Original clinical data preview:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   
2  !Sample_characteristics_ch1   
3  !Sample_characteristics_ch1   
4  !Sample_characteristics_ch1   

                                          GSM7498589  \
0                                tissue: Whole blood   
1                                       gender: Male   
2                                            age: 63   
3                              cardiopathy: ischemic   
4  cardiac rhythm at start of the study: Sinus rh...   

                                          GSM7498590  \
0                                tissue: Whole blood   
1                                       gender: Male   
2                                            age: 60   
3                              cardiopathy: ischemic   
4  cardiac rhythm at start of the study: Sinus rh...   

                                          GSM7498591  \
0                                

Data shape after handling missing values: (119, 19850)
For the feature 'Atrial_Fibrillation', the least common label is '1.0' with 13 occurrences. This represents 10.92% of the dataset.
Quartiles for 'Age':
  25%: 55.0
  50% (Median): 63.0
  75%: 68.0
Min: 37.0
Max: 79.0
For the feature 'Gender', the least common label is '0.0' with 32 occurrences. This represents 26.89% of the dataset.


Linked data saved to ../../output/preprocess/Atrial_Fibrillation/GSE235307.csv
